In [2]:
import numpy as np
import h5py

In [7]:
#let's extract our training, test and validation data
#method seen in [1] p21
with h5py.File('../HDF5/hdf5_uTHCD_compressed.h5', 'r') as hdf:
    base_items = list(hdf.items())
    print(f'Items in base dir: {base_items}')
    G1 = hdf.get('Train Data')
    G1_items = list(G1.items())
    print(f'items in group 1: {G1_items}')
    
    G2 = hdf.get('Test Data')
    G2_items = list(G2.items())
    print(f'items in group 2: {G2_items}')
    
    x_train = np.array(G1.get('x_train'))
    y_train = np.array(G1.get('y_train'))
    
    x_test = np.array(G2.get('x_test'))
    y_test = np.array(G2.get('y_test'))
    
    x_val = x_train[-7870:,:,:]
    y_val = y_train[-7870:]
    
    x_train = x_train[:-7870,:,:]
    y_train = y_train[:-7870]
    
x_train

Items in base dir: [('Test Data', <HDF5 group "/Test Data" (2 members)>), ('Train Data', <HDF5 group "/Train Data" (2 members)>)]
items in group 1: [('x_train', <HDF5 dataset "x_train": shape (62870, 64, 64), type "|u1">), ('y_train', <HDF5 dataset "y_train": shape (62870,), type "<i8">)]
items in group 2: [('x_test', <HDF5 dataset "x_test": shape (28080, 64, 64), type "|u1">), ('y_test', <HDF5 dataset "y_test": shape (28080,), type "<i8">)]


array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ...,   0, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       ...,

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ...,   0, 255, 255